In [1]:
import torch
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from gtts import gTTS
import timm
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline

In [3]:
custom_model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp8/weights/best.pt', force_reload = True)
model_type = "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\Admin/.cache\torch\hub\master.zip
YOLOv5  2022-6-27 Python-3.8.8 torch-1.9.1+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7255552 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\Admin/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Admin/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


Loading weights:  None


In [4]:
custom_model.names = ['NULL','person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eyeglasses' ,'handbag', 'tie', 
        'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard','tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk' ,'toilet', 'door', 'tv', 'laptop',
         'mouse', 'remote', 'keyboard', 'cell phone','microwave', 'oven', 'toaster', 'sink', 'refrigerator','blender','book', 'clock', 'vase', 'scissors', 'teddy bear','hair drier', 'toothbrush', 'hair brush']

In [5]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in C:\Users\Admin/.cache\torch\hub\intel-isl_MiDaS_master


In [28]:
filename = 'yolov5/NUS_Cycle.jpeg'

In [29]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(
        3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
        (pad): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
      )
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel

In [8]:
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

input_batch = transform(img).to(device)

In [9]:
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()

In [23]:
def split_depthmap(depthmap):
    
    width = depthmap.shape[1]
    height = depthmap.shape[0]
    
    depthmap = depthmap[:int(height*0.7),:]
    
    max_depth_point = (np.where(depthmap== np.max(depthmap)))[1][0]
    
    if max_depth_point <= (width//3):
        return "left"
    elif (width//3) < max_depth_point <= (width * 2/3):
        return "middle"
    else:
        return "right"
       

In [24]:
def split_image(image, side):
    
    width = image.shape[1]
    height = image.shape[0]
    
    left_side = image[:,:width//3,:]
    middle = image[:,width//3:2*(width//3),:]
    right_side = image[:,2*(width//3):,:]
    
    if side == "left":
        return left_side
    elif side == "middle":
        return middle
    else:
        return right_side

In [25]:
def give_alert(img, depthmap):
    
    aoi_region = split_depthmap(depthmap)     ###WHICH SIDE OBJECT IS ON
    
    aoi_image = split_image(img, aoi_region)
    
    results = custom_model(aoi_image)
    
    if results.pandas().xyxy[0].empty:
        ##GIVE AUDIO ALERT FOR UNKNOWN OBJECT at detected side
        to_return = "Unknown object at {} side".format(aoi_region) 
        return to_return
    else:
        obj = results.pandas().xyxy[0].iloc[0]['name']
        ## GIVE AUDIO ALERT FOR obj and side    
        to_return = "{} at {} side".format(obj, aoi_region)
        return to_return

In [26]:
def pipeline(img_path):
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)
    
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()
    
    return give_alert(img, output)

In [30]:
filename = 'yolov5/NUS_Cycle.jpeg'
pipeline(filename)

'bicycle object at right side'

In [4]:
model = torch.load('D://Downloads//yolov5s.pt')

ModuleNotFoundError: No module named 'models'